In [31]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import os

def load_model(name, weight_path, color):
    p = Path(weight_path)
    if not p.exists():
        print(f"Model missing, skipped: {name} -> {weight_path}")
        return None
    return {"name": name, "model": YOLO(str(p)), "color": color}

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    inter_w = max(0, inter_x2 - inter_x1)
    inter_h = max(0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area_a = max(0, ax2 - ax1) * max(0, ay2 - ay1)
    area_b = max(0, bx2 - bx1) * max(0, by2 - by1)
    union = area_a + area_b - inter + 1e-9
    return inter / union

def global_nms(dets, iou_thr=0.50):
    dets = sorted(dets, key=lambda d: d["conf"], reverse=True)
    keep = []
    for d in dets:
        ok = True
        for k in keep:
            if iou_xyxy(d["bbox"], k["bbox"]) >= iou_thr:
                ok = False
                break
        if ok:
            keep.append(d)
    return keep

candidates = [
    ("RoadWorks",     "/home/imadb/runs/detect/RoadWorks2/weights/best.pt",                   (0, 255, 0)),
    ("SpeedBumps",    "/home/imadb/runs/detect/SpeedBumps/weights/best.pt",                   (255, 0, 0)),
    ("StopSign",      "/home/imadb/runs/detect/StopSign/weights/best.pt",                     (0, 0, 255)),
    ("RoadDamage",    "/home/imadb/Ghiles/Projet_Ghiles/runs/detect/train4/weights/best.pt",  (255, 255, 0)),
    ("RoadDebris1",   "/home/imadb/runs/detect/RoadDebris1/weights/best.pt",                  (255, 0, 255)),
    ("RoadObstacle",  "/home/imadb/runs/detect/road_obstacle_yolov8l/weights/best.pt",        (0, 128, 255)),
]

models = []
for name, path, color in candidates:
    m = load_model(name, path, color)
    if m:
        models.append(m)

assert len(models) > 0, "No valid models loaded"

img_path = "/home/imadb/Ghiles/debris2/6.JPG"
assert os.path.exists(img_path), f"Image not found: {img_path}"

CONF = 0.4
IOU = 0.70
NMS_IOU = 0.50

all_detections = []

for item in models:
    model_name = item["name"]
    model = item["model"]
    color = item["color"]

    results = model(img_path, conf=CONF, iou=IOU, device=0, verbose=False)[0]

    print(f"\n{model_name} detections")
    if results.boxes is None or len(results.boxes) == 0:
        print("none")
        continue

    for box in results.boxes:
        cls = int(box.cls)
        cls_name = results.names[cls]
        conf = float(box.conf)
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

        print(f"{cls_name} {conf:.2f} ({x1},{y1},{x2},{y2})")

        all_detections.append({
            "model": model_name,
            "class": cls_name,
            "conf": conf,
            "bbox": (x1, y1, x2, y2),
            "color": color
        })

filtered = global_nms(all_detections, iou_thr=NMS_IOU)

img = cv2.imread(img_path)
assert img is not None, "Image read failed"

for d in filtered:
    x1, y1, x2, y2 = d["bbox"]
    color = d["color"]
    label = f"{d['model']}:{d['class']} {d['conf']:.2f}"

    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        img,
        label,
        (x1, max(15, y1 - 5)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.55,
        color,
        2,
        cv2.LINE_AA
    )

out_path = "result_all_models_nms.jpg"
cv2.imwrite(out_path, img)

print(f"\nDetections before NMS: {len(all_detections)}")
print(f"Detections after  NMS: {len(filtered)}")
print("Saved:", out_path)

print("\nSummary")
if not filtered:
    print("no detections")
else:
    filtered = sorted(filtered, key=lambda d: d["conf"], reverse=True)
    for d in filtered:
        x1, y1, x2, y2 = d["bbox"]
        print(f"{d['model']} | {d['class']} | {d['conf']:.2f} | ({x1},{y1},{x2},{y2})")



RoadWorks detections
cones 0.87 (342,325,357,359)
cones 0.87 (325,323,336,350)
cones 0.55 (334,309,343,328)

SpeedBumps detections
none

StopSign detections
none

RoadDamage detections
none

RoadDebris1 detections
object 0.78 (309,469,427,583)

RoadObstacle detections
road_block 0.57 (314,312,364,359)
road_block 0.55 (330,312,365,361)

Detections before NMS: 6
Detections after  NMS: 5
Saved: result_all_models_nms.jpg

Summary
RoadWorks | cones | 0.87 | (342,325,357,359)
RoadWorks | cones | 0.87 | (325,323,336,350)
RoadDebris1 | object | 0.78 | (309,469,427,583)
RoadObstacle | road_block | 0.57 | (314,312,364,359)
RoadWorks | cones | 0.55 | (334,309,343,328)


In [30]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import os

def load_model(name, weight_path, color):
    p = Path(weight_path)
    if not p.exists():
        print(f"Model missing, skipped: {name} -> {weight_path}")
        return None
    return {"name": name, "model": YOLO(str(p)), "color": color}

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    inter_w = max(0, inter_x2 - inter_x1)
    inter_h = max(0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area_a = max(0, ax2 - ax1) * max(0, ay2 - ay1)
    area_b = max(0, bx2 - bx1) * max(0, by2 - by1)
    union = area_a + area_b - inter + 1e-9
    return inter / union

def global_nms(dets, iou_thr=0.50):
    dets = sorted(dets, key=lambda d: d["conf"], reverse=True)
    keep = []
    for d in dets:
        ok = True
        for k in keep:
            if iou_xyxy(d["bbox"], k["bbox"]) >= iou_thr:
                ok = False
                break
        if ok:
            keep.append(d)
    return keep

def clamp_bbox(x1, y1, x2, y2, w, h):
    x1 = max(0, min(int(x1), w - 1))
    y1 = max(0, min(int(y1), h - 1))
    x2 = max(0, min(int(x2), w - 1))
    y2 = max(0, min(int(y2), h - 1))
    # assure x1<=x2, y1<=y2
    if x2 < x1: x1, x2 = x2, x1
    if y2 < y1: y1, y2 = y2, y1
    return x1, y1, x2, y2

# Config 
candidates = [
    ("RoadWorks",     "/home/imadb/runs/detect/RoadWorks2/weights/best.pt",                   (0, 255, 0)),
    ("SpeedBumps",    "/home/imadb/runs/detect/SpeedBumps/weights/best.pt",                   (255, 0, 0)),
    (#"StopSign",      "/home/imadb/runs/detect/StopSign/weights/best.pt",                     (0, 0, 255)),
    ("RoadDamage",    "/home/imadb/Ghiles/Projet_Ghiles/runs/detect/train4/weights/best.pt",  (255, 255, 0)),
    (#"RoadDebris1",   "/home/imadb/runs/detect/RoadDebris1/weights/best.pt",                  (255, 0, 255)),
    ("RoadObstacle",  "/home/imadb/runs/detect/road_obstacle_yolov8l/weights/best.pt",        (0, 128, 255)),
]

#  Paramètres YOLO
CONF = 0.20      
IOU = 0.65
NMS_IOU = 0.50
DEVICE = 0       # 0 = GPU, sinon "cpu"

# Vidéos
input_video = "13588981_3840_2160_30fps.mp4"   
output_video = "output_detected.avi"          

#  Load models
models = []
for name, path, color in candidates:
    m = load_model(name, path, color)
    if m:
        models.append(m)
assert len(models) > 0, "No valid models loaded"

# Video IO 
assert os.path.exists(input_video), f"Video not found: {input_video}"

cap = cv2.VideoCapture(input_video)
assert cap.isOpened(), "Could not open input video"

fps = cap.get(cv2.CAP_PROP_FPS)
if fps is None or fps <= 1:
    fps = 25.0  # fallback

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Codec robuste : XVID dans AVI
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = cv2.VideoWriter(output_video, fourcc, fps, (w, h))
assert writer.isOpened(), "Could not open output writer (try another codec)"

print(f"Input : {input_video}")
print(f"Output: {output_video} ({w}x{h} @ {fps:.2f} fps)")

# Process 
frame_id = 0
while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame_id += 1

    all_detections = []

    for item in models:
        model_name = item["name"]
        model = item["model"]
        color = item["color"]

        # inference
        results = model(frame, conf=CONF, iou=IOU, device=DEVICE, verbose=False)[0]

        if results.boxes is None or len(results.boxes) == 0:
            continue

        for box in results.boxes:
            cls = int(box.cls)
            cls_name = results.names[cls]
            conf = float(box.conf)
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

            x1, y1, x2, y2 = clamp_bbox(x1, y1, x2, y2, w, h)

            all_detections.append({
                "model": model_name,
                "class": cls_name,
                "conf": conf,
                "bbox": (x1, y1, x2, y2),
                "color": color
            })

    # NMS global (tous modèles)
    filtered = global_nms(all_detections, iou_thr=NMS_IOU)

    # dessin
    for d in filtered:
        x1, y1, x2, y2 = d["bbox"]
        color = d["color"]
        label = f"{d['model']}:{d['class']} {d['conf']:.2f}"

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            frame,
            label,
            (x1, max(15, y1 - 6)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.55,
            color,
            2,
            cv2.LINE_AA
        )

    writer.write(frame)

    # petit log toutes les 100 frames
    if frame_id % 100 == 0:
        print(f"Processed {frame_id} frames...")

cap.release()
writer.release()

print("Saved:", output_video)
print("Tip: open with VLC -> vlc output_detected.avi")


Input : 13588981_3840_2160_30fps.mp4
Output: output_detected.avi (3840x2160 @ 29.97 fps)
Processed 100 frames...
Processed 200 frames...
Processed 300 frames...
Processed 400 frames...
Processed 500 frames...
Saved: output_detected.avi
Tip: open with VLC -> vlc output_detected.avi
